<a href="https://colab.research.google.com/github/alyssa1996/whats-your-room/blob/main/whats_your_room.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
url = "https://docs.google.com/spreadsheets/d/1f73hTHz50lqkuXD3KLy0Al7yUK80JSqrC0iUH3Xakwg/htmlview#"
dfs = pd.read_html(url,encoding="UTF-8")

In [2]:
room_escape = dfs[0]
escape_data = pd.DataFrame(data=room_escape,index=room_escape.index[1:],columns=room_escape.columns[1:10])

escape_data =  escape_data.rename(columns=escape_data.iloc[0])
escape_data.drop(["번호"], axis=1, inplace=True)
escape_data.drop(escape_data.index[0:2], inplace=True) #[5,6], axis=0, inplace=True


# 활동성 열 데이터 가공
escape_data["활동성"] = escape_data["활동성"].fillna(0)
escape_data["활동성"] = escape_data["활동성"].replace({"●1": 1, "●2": 2, "●3" : 3})

#공포도 열 데이터 가공
escape_data["공포도"] = escape_data["공포도"].fillna(0)
escape_data["공포도"] = escape_data["공포도"].replace({"●0": 1,"●1": 2, "●2": 3, "●3" : 4})

#추천도 열 데이터 가공
# 사용하지 않는 점수의 추천도는 삭제
escape_data = escape_data[escape_data["추천도"].apply(lambda x: (x.find("("))) == 0]
escape_data = escape_data[escape_data["추천도"].apply(lambda x: (x.startswith("(9)") or x.startswith("(a)"))) == False]

recommendation_list = []
# 1. S가 들어가면 3로, A가 들어가면 2로, B가 들어가면 1로, C가 들어가면 0로 추천도 범위 통합
for key, value in enumerate(escape_data["추천도"]) :
  split_data = value.split(")")
  test = split_data[len(split_data)-1]
  if test.startswith("S"):
    recommendation_list.append(3)
  elif test.startswith("A"):
    recommendation_list.append(2)
  elif test.startswith("B"):
    recommendation_list.append(1)
  elif test.startswith("C"):
    recommendation_list.append(0)
rec_data = pd.Series(recommendation_list, index=list(range(3,258)))
escape_data["추천도"] = rec_data

# 난이도 열 데이터 가공 및 범위 조정
escape_data['난이도'] = pd.to_numeric(escape_data['난이도'], errors="coerce")
new_level = [0,5,7,9,11]

for index, value in enumerate(new_level):
  if index != len(new_level)-1:
    con1 = escape_data['난이도'] > value
    con2 = escape_data['난이도'] <= new_level[index+1]
    final_con = con1&con2
    escape_data['난이도']=escape_data['난이도'].mask(final_con,index+1)

escape_data

,지역,지점명,호점,테마명,난이도,추천도,공포도,활동성
3,강남,"방탈출, 단편선",NaN,사람들은 그것을 행복이라 부르기로 했다,2.0,3,1,0
4,부평,넥스트에디션,NaN,THISMAN Original,4.0,3,0,1
5,건대,솔버,2호점,디어 마르시,3.0,3,0,1
6,강남,제로월드,NaN,콜러,3.0,3,4,1
7,강남,제로월드,NaN,제로,3.0,3,4,2
...,...,...,...,...,...,...,...,...
253,강남,코드케이,NaN,미치광이 과학자 연쇄살인범,3.0,0,1,0
254,신촌,탐정크루,1호점,방과 방 사이,2.0,0,0,0
255,일산,명란젓코난,NaN,마녀의 초대장,2.0,0,0,0
256,부천,로스트템플,NaN,드라큘라 백작의 성,1.0,0,1,0


In [3]:
escape_data.to_csv('escape.csv',index = False) 

In [ ]:
escape_data["지역"].unique()

array(['강남', '부평', '건대', '홍대', '신촌', '잠실', '수원', '부천', '대구', '혜화', '신림',
       '분당', '성수', '인천', '대학로', '성남', '종각', '상수', '수유', '서면', '대전', '신논현',
       '서울대입구', '노량진', '일산', '구로'], dtype=object)

In [ ]:
escape_data["지역"].value_counts()

강남       70
홍대       51
건대       32
혜화       14
신촌       12
신림       11
잠실        9
대구        9
부평        8
부천        6
인천        5
수원        5
상수        3
신논현       3
분당        2
대학로       2
서면        2
대전        2
서울대입구     2
성수        1
성남        1
종각        1
수유        1
노량진       1
일산        1
구로        1
Name: 지역, dtype: int64

In [ ]:
escape_data["난이도"].value_counts()

7       63
8       55
6       48
9       28
5       23
4       15
10      10
Misc     7
11       5
3        1
Name: 난이도, dtype: int64

In [ ]:
escape_data["추천도"].value_counts()

B+      56
A+      53
A       49
S       28
B       23
S+      17
C+      11
C        5
F        4
X        4
Misc     4
S++      1
Name: 추천도, dtype: int64

In [ ]:
escape_data["공포도"].value_counts()

0    173
1     35
2     24
3     14
4      9
Name: 공포도, dtype: int64

In [ ]:
escape_data["활동성"].value_counts()

0    107
1     87
2     38
3     23
Name: 활동성, dtype: int64

In [ ]:
# sample_data = [지역, 난이도, 추천도, 공포도, 활동성]

test_data1 = {"지역" : ["강남"],"난이도": ["6","7","8"], "추천도":["A","A+","S", "S+", "S++"], "공포도":[0], "활동성":[0,1]}
test_data2 = {"지역" : ["신촌", "홍대"],"난이도": ["5","6","7"], "추천도":["S", "S+", "S++"], "공포도":[0], "활동성":[0,1]}
test_data3 = {"지역" : ["건대"],"난이도": ["7", "8", "9"], "추천도":["A", "A+","S", "S+", "S++"], "공포도":[0], "활동성":[1,2,3]}


recommended_room = pd.DataFrame(escape_data)
for key , value in test_data3.items():
  recommended_room = recommended_room[recommended_room[key].isin(value)]

recommended_room

,번호,지역,지점명,호점,테마명,난이도,추천도,공포도,활동성
5,[062],건대,솔버,2호점,디어 마르시,9,S+,0,1
40,[276],건대,넥스트에디션,보네르관,세렌디피티,7,S,0,2
56,[140],건대,솔버,2호점,무채색 인간,9,A+,0,1
58,[269],건대,머더파커,2호점,동창회,9,A+,0,1
60,[065],건대,넥스트에디션,1호점,다시 봄,8,A+,0,3
74,[046],건대,넥스트에디션,1호점,몬스터 : 10800,7,A+,0,2
77,[136],건대,머더파커,NaN,여행,7,A+,0,1
112,[126],건대,넥스트에디션,2호점,동화나라 수비대,8,A,0,1
123,[018],건대,솔버,1호점,콜드케이스,7,A,0,2
